# 第4章 LlamaIndex知识管理与信息检索

# 学习目标

1. 掌握 LlamalIndex 的特点和基本用法  
2. 掌握 LlamalIndex 内置的工具  
3. 如何用好SDK简化基于LLM的应用开发

1. 大语言模型开发框架的价值是什么？

SDK：Software Development Kit，它是一组软件工具和资源的集合，旨在帮助开发者创建、测试、部署和维护应用程序或软件。

所有开发框架（SDK）的核心价值，都是降低开发、维护成本。

大语言模型开发框架的价值，是让开发者可以更方便地开发基于大语言模型的应用。主要提供两类帮助：

1. 第三方能力抽象。比如LLM、向量数据库、搜索接口等  
2. 常用工具、方案封装  
3.底层实现封装。比如流式接口、超时重连、异步与并行等

好的开发框架，需要具备以下特点：

1. 可靠性、鲁棒性高  
2. 可维护性高  
3. 可扩展性高  
4. 学习成本低

举些通俗的例子：

- 与外部功能解依赖

比如可以随意更换LLM而不用大量重构代码  
- 更换三方工具也同理

- 经常变的部分要在外部维护而不是放在代码里

比如 Prompt 模板

- 各种环境下都适用

比如线程安全

- 方便调试和测试

- 至少要能感觉到用了比不用方便吧  
- 合法的输入不会引发框架内部的报错

划重点：选对了框架，事半功倍；反之，事倍功半。

什么是SDK? https://aws.amazon.com/cn/what-is/sdk/

SDK和API的区别是什么？https://aws.amazon.com/cn/compare/the-difference-between-sdk-and-api/
 
# 举个例子：使用SDK，4行代码实现一个简易的RAG系统

LlamaIndex默认的Embedding模型是

OpenAIEmbedding(model="text-embedding-adad002")
 


In [2]:
#! pip install llama-index 
#! llama-index-llms-dashscope 
#! llama-index-llms-openai-like 
#! llama-index-embeddings-dashscope

In [3]:
import os
from llama_index.core import Settings
from llama_index.llms.dashscope import DashScope,DashScopeGenerationModels
from llama_index.embeddings.dashscope import DashScopeEmbedding,DashScopeTextEmbeddingModels
from llama_index.core import  VectorStoreIndex,SimpleDirectoryReader

llm = DashScope(
    model_name=DashScopeGenerationModels.QWEN_MAX,
    api_key="sk-66bc27a6330f434f8751f8172a73064f"
)

Settings.llm = llm

embedding = DashScopeEmbedding(
    model_name=DashScopeTextEmbeddingModels.TEXT_EMBEDDING_V1,
    api_key="sk-66bc27a6330f434f8751f8172a73064f"
)
Settings.embed_model = embedding

documents = SimpleDirectoryReader("./data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()
response = query_engine.query("什么是 LangGraph")

print(response)


2025-12-08 22:18:26,565 - WARNING - Ignoring wrong pointing object 235 0 (offset 0)
2025-12-08 22:18:26,567 - WARNING - Ignoring wrong pointing object 236 0 (offset 0)


LangGraph 是一个用于构建有状态的多参与者应用程序的库，它利用 LLM（语言模型）来创建代理和多代理工作流。与其它 LLM 框架相比，LangGraph 提供了循环性、可控性和持久性的核心优势。它允许定义包含循环的流程，这对于大多数代理架构来说是必要的，并且提供了对应用流程和状态的细粒度控制。此外，LangGraph 支持自动保存状态以及暂停和恢复图执行的功能，这有助于错误恢复、“人机交互”工作流等场景。该框架还支持在每个节点产生输出时进行流式传输。简而言之，LangGraph 通过将代理工作流建模为图的方式，提供了一种强大而灵活的方法来开发复杂的语言代理应用。


# 2. LlamalIndex 介绍

官网标题：「Build AI Knowledge Assistants over your enterprise data」

- LlamaIndex 是一个为开发「知识增强」的大语言模型应用的框架（也就是 SDK）。知识增强，泛指任何在私有或特定领域数据基础上应用大语言模型的情况。例如：


![](./img/1.png)

# - Question-Answering Chatbots (也就是 RAG)

- Document Understanding and Extraction (文档理解与信息抽取)  
- Autonomous Agents that can perform research and take actions (智能体应用)  
- Workflow orchestrating single and multi-agent (编排单个或多个智能体形成工作流)
- LlamalIndex 有 Python 和 Typescript 两个版本, Python 版的文档相对更完善。  
- Python 文档地址: https://docs.llamaindex.ai/en/stable/  
- Python API 接口文档: https://docs.llamaindex.ai/en/stable/api.Reference/  
- TS 文档地址: https://ts.llamaindex.ai/

![](./img/2.png)


# 3.数据加载（Loading）

# 3.1、加载本地数据

SimpleDirectoryReader 是一个简单的本地文件加载器。它会遍历指定目录，并根据文件扩展名自动加载文件（文本内容）。

支持的文件类型：

```txt
.csv - comma-separated values   
.docx - Microsoft Word   
.epub - EPUB ebook format   
.hwp - Hangul Word Processor   
.ipynb - Jupyter Notebook   
.jpeg, .jpg - JPEG image   
.mbox - MBOX email archive   
.md - Markdown   
.mp3, .mp4 - audio and video   
.pdf - Portable Document Format   
.png - Portable Network Graphics   
.ppt, .pptm, .pptx - Microsoft PowerPoint
```

 

In [ ]:
import json
from reprlib import recursive_repr
from pydantic import BaseModel
from test1 import documents
from llama_index.core import SimpleDirectoryReader

def show_json(data):
    """
    用于展示 json 数据
    """
    if isinstance(data, str):
        obj = json.loads(data)
        print(json.dumps(obj, indent=4,ensure_ascii=False))
    elif isinstance(data, dict):
        print(json.dumps(data, indent=4, ensure_ascii=False))
    elif issubclass(type(data), BaseModel):
        print(json.dumps(data.dict, indent=4, ensure_ascii=False))

def show_list_obj(data):
    """
     用于展示一组对象
    """
    if isinstance(data, list):
        for item in data:
           show_list_obj(data)
    else:
        raise ValueError("Input is not a list")

reader = SimpleDirectoryReader(
    input_dir="./data",
    recursive=True,
    required_exts=[".pdf"]
)
documents = reader.load_data()

print(documents[0].text)
show_json(documents[0].json())

# 注意：对图像、视频、语音类文件，默认不会自动提取其中文字。如需提取，参考下面介绍的 Data Connectors。

默认的PDFReader效果并不理想，我们可以更换文件加载器

LlamaParse

首先，登录并从https://cloud.llamaindex.ai注册并获取api-key。

然后，安装该包：

![](./img/3.png)

![](./img/4.png)

# 4. 文本切分与解析 (Chunking)

为方便检索，我们通常把Document切分为Node。

在 LlamalIndex 中，Node 被定义为一个文本的「chunk」。

# 4.1、使用 TextSplitters 对文本做切分

例如：TokenTextSplitter 按指定 token 数切分文本



In [ ]:
from llama_index.core import Document
from llama_index.core.node_parser import TokenTextSplitter

node_parser = TokenTextSplitter(
    chunk_size=512,#每个 chunk 的最大长度
    chunk_overlap=200# chunk 之间重叠长度
)

nodes = node_parser.get_nodes_from_documents(documents,show_progress=True)

show_json(nodes[1].json())
show_json(nodes[2].json())

# LlamaIndex 提供了丰富的 TextSplitter，例如：

- SentenceSplitter：在切分指定长度的 chunk 同时尽量保证句子边界不被切断；(用的最多)  
- CodeSplitter：根据 AST（编译器的抽象句法树）切分代码，保证代码功能片段完整；  
- SemanticSplitterNodeParser：根据语义相关性对将文本切分为片段。


![](./img/5.png)

# 5. 索引（Indexing）与检索（Retrieval）

基础概念：在「检索」相关的上下文中，「索引」即 index，通常是指为了实现快速检索而设计的特定「数据结构」。

索引的具体原理与实现不是本课程的教学重点, 感兴趣的同学可以参考: 传统索引、向量索引

# 5.1、向量检索

1. VectorStoreIndex 直接在内存中构建一个 Vector Store 并建索引



In [7]:

import json
from reprlib import recursive_repr

from llama_index.core.base.embeddings.base import similarity
from llama_index.core.node_parser import TokenTextSplitter
from openai import vector_stores
from pydantic import BaseModel

from test1 import documents


def show_json(data):
    """
    用于展示 json 数据
    """
    if isinstance(data, str):
        obj = json.loads(data)
        print(json.dumps(obj, indent=4,ensure_ascii=False))
    elif isinstance(data, dict):
        print(json.dumps(data, indent=4, ensure_ascii=False))
    elif issubclass(type(data), BaseModel):
        print(json.dumps(data.dict, indent=4, ensure_ascii=False))

def show_list_obj(data):
    """
     用于展示一组对象
    """
    if isinstance(data, list):
        for item in data:
           show_list_obj(data)
    else:
        raise ValueError("Input is not a list")

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

reader = SimpleDirectoryReader(
    input_dir="./data",
    recursive=True,
    required_exts=[".pdf"]
)
documents = reader.load_data()

node_parser = TokenTextSplitter(
    chunk_size=512,chunk_overlap=200
)

nodes = node_parser.get_nodes_from_documents(documents,show_progress=True)

# 构建 index 默认是在内存中
index = VectorStoreIndex(nodes)

vector_retriver = index.as_retriever(
     similarity_top_k = 2
)
# 检索
result = vector_retriver.retrieve("LangGraph是什么")

print(result[0].text)

2025-12-08 23:09:04,994 - WARNING - Ignoring wrong pointing object 235 0 (offset 0)
2025-12-08 23:09:04,995 - WARNING - Ignoring wrong pointing object 236 0 (offset 0)
Parsing nodes: 100%|█████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 315.58it/s]


LangGraph 应用程序的开发、部署、调试和监控： LangGraph 服务器（ API）、 
LangGraph SDK（ API 客户端）、 LangGraph CLI（构建服务器的命令行工具）、 LangGraph 
Studio（用户界面 /调试器）。 
主要功能  
循环和分支 ：在您的应用程序中实现循环和条件语句。 
持久性 ：在图中的每个步骤之后自动保存状态。在任何时候暂停和恢复图执行以支持错误恢复、 
“人机交互 ”工作流、时间旅行等等。 
“人机交互 ”：中断图执行以批准或编辑代理计划的下一个动作。 
流支持 ：在每个节点产生输出时流式传输输出（包括令牌流式传输）。


![](./img/5.png)

# 5.2、更多索引与检索方式

LlamaIndex 内置了丰富的检索机制，例如：

- 关键字检索

- BM25Retriever：基于tokenizer实现的BM25经典检索算法  
- KeywordTableGPTRetriever：使用GPT提取检索关键字  
- KeywordTableSimpleRetriever：使用正则表达式提取检索关键字  
- KeywordTableRAKERetriever：使用RAKE算法提取检索关键字（有语言限制）

- RAG-Fusion QueryFusionRetriever  
- 还支持 KnowledgeGraph、SQL、Text-to-SQL 等等

# 5.3、检索后处理

LlamaIndex 的 Node Postprocessors 提供了一系列检索后处理模块。

例如：我们可以用不同模型对检索后的 Nodes 做重排序


![](./img/7.png)

![](./img/8.png)

![](./img/9.png)

![](./img/10.png)


# 7. 底层接口：Prompt、LLM与Embedding

# 7.1 Prompt模板

PromptTemplate 定义提示词模板

In [8]:
from llama_index.core import PromptTemplate

prompt = PromptTemplate('写一个关于{topic}的笑话')

prompt.format(topic="小米")



'写一个关于小米的笑话'

# ChatPromptTemplate

In [10]:
from  llama_index.core.llms import ChatMessage,MessageRole
from  llama_index.core import ChatPromptTemplate

chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content="你叫{name},你必须根据用户提供的上下文回答问题",
    ),
     ChatMessage(
        role=MessageRole.USER,
        content=(
            "已知上下文: \n"\
            "{context}\n\n"\
            "问题:{question}"
        )
    ),
]

text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

print(
    text_qa_template.format(
        name="小明",
        context="这是一个测试",
        question="LangGraph 是什么"
    )
)




system: 你叫小明,你必须根据用户提供的上下文回答问题
user: 已知上下文: 
这是一个测试

问题:LangGraph 是什么
assistant: 


![](./img/11.png)

![](./img/12.png)

![](./img/13.png)


# 8. 基于 LlamaIndex 实现一个功能较完整的 RAG 系统

# 功能要求：

加载指定目录的文件  
- 支持 RAG-Fusion  
- 使用 Qdrant 向量数据库，并持久化到本地  
- 支持检索后排序  
- 支持多轮对话






In [12]:
# !pip install -U qdrant_client 
# !pip install llama-index-vector-stores-qdrant

In [ ]:
from openai.types import Embedding
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams,Distance
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,get_response_synthesizer
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import Settings
from llama_index.core import StorageContext
from llama_index.core.postprocessor import LLMRerank,SimilarityPostprocessor
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.chat_engine import CondenseQuestionChatEngine
from llama_index.llms.dashscope import DashScope,DashScopeGenerationModels
from llama_index.embeddings.dashscope import DashScopeEmbedding,DashScopeTextEmbeddingModels

EMBEDDING_DIM=1536
COLLECTION_NAME="full_demo"
PATH="./qdrant_db"

client = QdrantClient(path=PATH)


llm = DashScope(
    model_name=DashScopeGenerationModels.QWEN_MAX,
    api_key="sk-66bc27a6330f434f8751f8172a73064f"
)

Settings.llm = llm

embedding = DashScopeEmbedding(
    model_name=DashScopeTextEmbeddingModels.TEXT_EMBEDDING_V1,
    api_key="sk-66bc27a6330f434f8751f8172a73064f"
)
Settings.embed_model = embedding

# 2 指定全局文档处理的 Ingestion Pipeline
Settings.transformations = [SentenceSplitter(chunk_size=512,chunk_overlap=200)]
# 3 加载本地文档
documents = SimpleDirectoryReader("./data").load_data()

if client.collection_exists(collection_name=COLLECTION_NAME):
    client.delete_collection(collection_name=COLLECTION_NAME)

# 4 创建 collection
client.create_collection(COLLECTION_NAME,
                         vectors_config=VectorParams(
                           size=EMBEDDING_DIM,distance=Distance.COSINE
                         )
                )

# 5 创建 Vector Store
vectore_store = QdrantVectorStore(
    client=client,collection_name=COLLECTION_NAME,
)
# 6 指定 Vectore Store 的 Store用于 index
storage_context = StorageContext.from_defaults(vector_store=vectore_store)
index = VectorStoreIndex.from_documents(documents,storage_context=storage_context)

# 7定义检索后排序模型
reranker = LLMRerank(top_n=2)
# 最终打分低于 0.6的文档被过滤掉
sp = SimilarityPostprocessor(similarity_cutoff=0.6)

# 8 定义 Rag fusion 检索器
fusion_retriever = QueryFusionRetriever(
    [index.as_retriever()],
    similarity_top_k=5 ,#检索召回 top key 结果
    num_queries=3,#生成 query 数
    use_async=False,
    query_gen_prompt="",#可以自定义 query 生成prompt 模板
)

# 9 构建单论 query engine
query_engine = RetrieverQueryEngine.from_args(
    retriever=fusion_retriever,
    node_postprocessors=[reranker],
    response_synthesizer=get_response_synthesizer(
        response_mode=ResponseMode.REFINE
    )
)
# 10 对话引擎
chat_engine = CondenseQuestionChatEngine.from_defaults(
    query_engine=query_engine,
    #condense_question_prompt="",#可自定义 chat message prompt
)


# 测试多轮对话
# User :  LangGraph 是什么
while True:
    question = input("User: ")
    if question.strip() == "e":
        break
    response = chat_engine.chat(question)
    print(response)
